In [ ]:
#include <WiFi.h>
#include <HTTPClient.h>
#include <OneWire.h>
#include <DallasTemperature.h>

const char* ssid = "TU_WIFI_AQUÍ";
const char* password = "TU_PASSWORD_AQUÍ";
String GOOGLE_SCRIPT_URL = "https://script.google.com/macros/s/TU_ID_AQUÍ/exec";

// --- CONFIGURACIÓN SENSORES ---
const int pinDatos = 4; // Cable amarillo de los sensores
OneWire oneWire(pinDatos);
DallasTemperature sensores(&oneWire);

// --- CONTROL DE TIEMPO (ANALÍSTICA) ---
unsigned long ultimoEnvio = 0;
const long intervaloEnvio = 60000; // Envío cada 1 minuto (ideal para tendencias)

void setup() {
  Serial.begin(115200);
  sensores.begin();
  
  WiFi.begin(ssid, password);
  Serial.print("Conectando a WiFi");
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }
  Serial.println("\n[INFO] WiFi Conectado con éxito.");
}

void loop() {
  unsigned long tiempoActual = millis();

  // Lógica de muestreo periódico
  if (tiempoActual - ultimoEnvio >= intervaloEnvio) {
    ultimoEnvio = tiempoActual;
    
    sensores.requestTemperatures(); 
    
    // Lectura de sensores (0 = Interior, 1 = Exterior)
    float t_conge = sensores.getTempCByIndex(0);
    float t_amb = sensores.getTempCByIndex(1);

    // Validación de integridad de datos (Evita guardar errores en el Excel)
    if (t_conge != -127.00 && t_amb != -127.00) {
      enviarDatosAGoogle(t_conge, t_amb);
    } else {
      Serial.println("[ERROR] Falla en lectura de sensores. Verifique cableado.");
    }
  }
}

void enviarDatosAGoogle(float t1, float t2) {
  if (WiFi.status() == WL_CONNECTED) {
    HTTPClient http;
    
    // Construcción de la URL con parámetros para el Analista
    // t_conge -> Sensor dentro de la cámara
    // t_amb   -> Sensor ambiente
    String urlFinal = GOOGLE_SCRIPT_URL + "?t_conge=" + String(t1) + "&t_amb=" + String(t2);
    
    http.begin(urlFinal.c_str());
    http.setFollowRedirects(HTTPC_STRICT_FOLLOW_REDIRECTS);
    
    int httpResponseCode = http.GET();
    
    if (httpResponseCode > 0) {
      Serial.print("[EXITO] Datos enviados. T1: "); Serial.print(t1);
      Serial.print(" | T2: "); Serial.println(t2);
    } else {
      Serial.print("[ERROR] Código de respuesta HTTP: "); Serial.println(httpResponseCode);
    }
    http.end();
  } else {
    Serial.println("[ERROR] WiFi desconectado.");
  }
}